In [9]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf  # Note: keras is not needed for PySoap to work

import PyNetwork

In [10]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(*x_train.shape, 1) / 255
x_test = x_test.reshape(*x_test.shape, 1) / 255

labels = np.eye(10)
y_train = labels[y_train]
y_test = labels[y_test]

In [15]:
model = PyNetwork.Sequential()

model.add( PyNetwork.layers.Input((28, 28, 1)) )
# model.add( PyNetwork.layers.Conv_2D(3, (3, 3), 1, activation_function='relu') )
model.add( PyNetwork.layers.Flatten() )
model.add( PyNetwork.layers.Dense(100, activation_function='relu', l2=0.001) )
model.add( PyNetwork.layers.BatchNorm() )
model.add( PyNetwork.layers.Dense(10, activation_function='softmax') )

# optimizer = PySoap.optimizers.SGD(learning_rate=0.01, momentum=0.001)
model.build(loss_function='cross_entropy', optimizer="adam", metrics = 'accuracy')

In [46]:
model = PyNetwork.Sequential()

model.add( PyNetwork.layers.Input((28, 28, 1)) )
model.add( PyNetwork.layers.Conv_2D(3, (3, 3), 1, activation_function='relu') )
model.add( PyNetwork.layers.Flatten() )
model.add( PyNetwork.layers.Dense(100, activation_function='relu', l2=0.0001) )
model.add( PyNetwork.layers.BatchNorm() )
model.add( PyNetwork.layers.Dense(10, activation_function='softmax') )

# optimizer = PySoap.optimizers.SGD(learning_rate=0.01, momentum=0.001)
model.build(loss_function='cross_entropy', optimizer="adam", metrics = 'accuracy')

In [47]:
model.summary()

Input                 :    Input Shape  (None, 28, 28, 1)
Conv 2D 3 x (3, 3)    :    Output Shape (None, 26, 26, 3)
Flatten               :    Output Shape (None, 2028)
Dense (100,)          :    Output Shape (None, 100)
Batch Norm            :    Output Shape (None, 100)
Dense (10,)           :    Output Shape (None, 10)


In [48]:
%%time
model.train(x_train, y_train, epochs=4, batch_size=128, verbose=True)

Training on 60000 samples
Epoch 1/4
cross_entropy: 0.2513 - accuracy: 0.9219
Training on 60000 samples
Epoch 2/4
cross_entropy: 0.1775 - accuracy: 0.9609
Training on 60000 samples
Epoch 3/4
cross_entropy: 0.1314 - accuracy: 0.9766
Training on 60000 samples
Epoch 4/4
cross_entropy: 0.1350 - accuracy: 0.9609
CPU times: user 2min 46s, sys: 50.7 s, total: 3min 37s
Wall time: 22.8 s


In [49]:
model.evaluate(x_test, y_test)

'cross_entropy: 0.1750 - accuracy: 0.9482'